In [ ]:
# ==========================================================
# 1️⃣ Install package & setup environment
# ==========================================================
!pip install /kaggle/input/lmsys-packages/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/lmsys-packages/xformers-0.0.24042abc8.d20240802-cp310-cp310-linux_x86_64.whl

!cp -r /kaggle/input/lmsys-modules-0805 human_pref

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# ==========================================================
# 2️⃣ prepare_test_file.py
# ==========================================================
with open("prepare_test_file.py", "w") as f:
    f.write('''\
import pandas as pd

df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
df["winner_model_a"] = 1
df["winner_model_b"] = 0
df["winner_tie"] = 0
df.to_parquet("test.parquet", index=False)

df["response_a"], df["response_b"] = df["response_b"], df["response_a"]
df.to_parquet("test_swap.parquet", index=False)
''')

!python prepare_test_file.py

# ==========================================================
# 3️⃣ predict_m0.py (Gemma2)
# ==========================================================
with open("predict_m0.py", "w") as f:
    f.write('''\
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer
from xformers.ops.fmha.attn_bias import BlockDiagonalCausalMask
from human_pref.models.modeling_gemma2 import Gemma2ForSequenceClassification
from human_pref.data.processors import ProcessorPAB
from human_pref.data.dataset import LMSYSDataset
from human_pref.data.collators import VarlenCollator, ShardedMaxTokensCollator
from human_pref.utils import to_device

model_name_or_path = "/kaggle/input/lmsys-checkpoints-0-0805"
csv_path = "test.parquet"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
processor = ProcessorPAB(tokenizer=tokenizer, max_length=4096, support_system_role=False)
dataset = LMSYSDataset(csv_file=csv_path, query=None, processor=processor, include_swap=False, is_parquet=True)
dataloader = DataLoader(dataset, batch_size=80, num_workers=4, collate_fn=ShardedMaxTokensCollator(max_tokens=8192, base_collator=VarlenCollator()))

num_hidden_layers = 42
device_map = {"model.embed_tokens": "cuda:0", "model.norm": "cuda:1", "score": "cuda:1"}
for i in range(num_hidden_layers // 2): device_map[f"model.layers.{i}"] = "cuda:0"
for i in range(num_hidden_layers // 2, num_hidden_layers): device_map[f"model.layers.{i}"] = "cuda:1"

model = Gemma2ForSequenceClassification.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map=device_map)

config = model.config
dim = config.head_dim
inv_freq = 1.0 / (config.rope_theta ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
inv_freq0 = inv_freq.to("cuda:0"); inv_freq1 = inv_freq.to("cuda:1")

is_first = True
outs = []
for batch in tqdm(dataloader):
    for micro_batch in batch:
        input_ids = to_device(micro_batch["input_ids"], "cuda:0")
        seq_info = dict(
            cu_seqlens=micro_batch["cu_seqlens"],
            position_ids=micro_batch["position_ids"],
            max_seq_len=micro_batch["max_seq_len"],
            attn_bias=BlockDiagonalCausalMask.from_seqlens(micro_batch["seq_lens"]),
        )
        seq_info = to_device(seq_info, "cuda:0")
        if is_first:
            with torch.no_grad(), torch.cuda.amp.autocast():
                prev_hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            is_first = False
            prev_seq_info, prev_hidden_states = to_device([seq_info, prev_hidden_states], "cuda:1")
            continue
        with torch.no_grad(), torch.cuda.amp.autocast():
            logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
            hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            prev_seq_info, prev_hidden_states = to_device([seq_info, hidden_states], "cuda:1")
            outs.append(logits.cpu())

with torch.no_grad(), torch.cuda.amp.autocast():
    logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
    outs.append(logits.cpu())

pred = torch.cat(outs, dim=0)
prob = pred.softmax(-1)
print(dataset.evaluate(prob.numpy()))
np.save('prob_m0.npy', prob)
''')

!python predict_m0.py

# ==========================================================
# 4️⃣ predict_m3.py (Llama3)
# ==========================================================
with open("predict_m3.py", "w") as f:
    f.write('''\
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer
from xformers.ops.fmha.attn_bias import BlockDiagonalCausalMask
from human_pref.models.modeling_llama import LlamaForSequenceClassification
from human_pref.data.processors import ProcessorPAB
from human_pref.data.dataset import LMSYSDataset
from human_pref.data.collators import VarlenCollator, ShardedMaxTokensCollator
from human_pref.utils import to_device

model_name_or_path = "/kaggle/input/lmsys-checkpoints-3-0805"
csv_path = "test_swap.parquet"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
processor = ProcessorPAB(tokenizer=tokenizer, max_length=4096, support_system_role=True)
dataset = LMSYSDataset(csv_file=csv_path, query=None, processor=processor, include_swap=False, is_parquet=True)
dataloader = DataLoader(dataset, batch_size=80, num_workers=4, collate_fn=ShardedMaxTokensCollator(max_tokens=8192, base_collator=VarlenCollator()))

num_hidden_layers = 32
device_map = {"model.embed_tokens": "cuda:0", "model.norm": "cuda:1", "score": "cuda:1"}
for i in range(num_hidden_layers // 2): device_map[f"model.layers.{i}"] = "cuda:0"
for i in range(num_hidden_layers // 2, num_hidden_layers): device_map[f"model.layers.{i}"] = "cuda:1"

model = LlamaForSequenceClassification.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map=device_map)
config = model.config
dim = config.hidden_size // config.num_attention_heads
inv_freq = 1.0 / (config.rope_theta ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
inv_freq0 = inv_freq.to("cuda:0"); inv_freq1 = inv_freq.to("cuda:1")

is_first = True
outs = []
for batch in tqdm(dataloader):
    for micro_batch in batch:
        input_ids = to_device(micro_batch["input_ids"], "cuda:0")
        seq_info = dict(
            cu_seqlens=micro_batch["cu_seqlens"],
            position_ids=micro_batch["position_ids"],
            max_seq_len=micro_batch["max_seq_len"],
            attn_bias=BlockDiagonalCausalMask.from_seqlens(micro_batch["seq_lens"]),
        )
        seq_info = to_device(seq_info, "cuda:0")
        if is_first:
            with torch.no_grad(), torch.cuda.amp.autocast():
                prev_hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            is_first = False
            prev_seq_info, prev_hidden_states = to_device([seq_info, prev_hidden_states], "cuda:1")
            continue
        with torch.no_grad(), torch.cuda.amp.autocast():
            logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
            hidden_states = model.forward_part1(input_ids, seq_info, inv_freq0)
            prev_seq_info, prev_hidden_states = to_device([seq_info, hidden_states], "cuda:1")
            outs.append(logits.cpu())

with torch.no_grad(), torch.cuda.amp.autocast():
    logits = model.forward_part2(prev_hidden_states, prev_seq_info, inv_freq1)
    outs.append(logits.cpu())

pred = torch.cat(outs, dim=0)
prob = pred.softmax(-1)
print(dataset.evaluate(prob.numpy()))
np.save('prob_m3.npy', prob)
''')

!python predict_m3.py

# ==========================================================
# 5️⃣ ensemble.py — final prediction
# ==========================================================
with open("ensemble.py", "w") as f:
    f.write('''\
import numpy as np
import pandas as pd

df = pd.read_parquet("test.parquet")
prob_m0 = np.load("prob_m0.npy")
prob_m3 = np.load("prob_m3.npy")[:, [1, 0, 2]]

preds = np.average([prob_m0, prob_m3], axis=0, weights=[0.55, 0.45])
sub = pd.DataFrame({
    "id": df["id"],
    "winner_model_a": preds[:, 0],
    "winner_model_b": preds[:, 1],
    "winner_tie": preds[:, 2],
})
sub.to_csv("submission.csv", index=False)
print(sub.head())
''')

!python ensemble.py
